# ====--------任务二--第一组--------==== #

In [1]:
p = (1 << 31) - 1
q = (1 << 61) - 1

In [2]:
def is_prime(n: int) -> bool:
    """判断是否为素数"""
    if n <= 1:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    max_divisor = int(n ** 0.5) + 1
    for i in range(3, max_divisor, 2):
        if n % i == 0:
            return False
    return True

In [3]:
print("验证 2^31 - 1 是否为素数：", is_prime(p) == True)
print("验证 2^61 - 1 是否为素数：", is_prime(q) == True)

验证 2^31 - 1 是否为素数： True
验证 2^61 - 1 是否为素数： True


In [4]:
import random

def gcd(a: int, b: int) -> int:
    a = abs(a)
    b = abs(b)
    if a == 0:
        return b
    if b == 0:
        return a
    while b != 0:
        a, b = b, a % b
    return a

def random_element_from_multiplicative_group(n: int) -> int:
    """从模n的乘法群（Z*n）中随机选取一个元素"""
    if n <= 1:
        raise ValueError("模数必须大于1")
    
    top = 100
    times = 0
    while times < top:
        candidate = random.randint(1, n - 1)
        if gcd(candidate, n) == 1:
            return candidate
    print("取值失败")

In [5]:
ele_p = random_element_from_multiplicative_group(p)
ele_q = random_element_from_multiplicative_group(q)
print("从 Z*p 中随机取的元素为：", ele_p)
print("从 Z*q 中随机取的元素为：", ele_q)

从 Z*p 中随机取的元素为： 374748896
从 Z*q 中随机取的元素为： 2075071169903309045


In [6]:
def extended_gcd(a: int, b: int) -> tuple[int, int, int]:
    """扩展欧几里得算法，计算 gcd(a, b) 以及满足 ax + by = gcd(a, b) 的系数 x 和 y"""
    if b == 0:
        return (a, 1, 0)
    else:
        g, x, y = extended_gcd(b, a % b)
        return (g, y, x - (a // b) * y)

def chinese_remainder_theorem(p: int, q: int, ele_p: int, ele_q: int) -> int:
    """中国剩余定理求解函数"""
    ele_p = ele_p % p
    ele_q = ele_q % q
    g, m, n = extended_gcd(q, p)
    if g != 1:
        raise ValueError("p和q必须互质")
    q_inv_mod_p = m % p
    g, m, n = extended_gcd(p, q)
    if g != 1:
        raise ValueError("p和q必须互质")
    p_inv_mod_q = m % q
    x = (ele_p * q * q_inv_mod_p + ele_q * p * p_inv_mod_q) % (p * q)
    return x

In [7]:
res = chinese_remainder_theorem(p, q, ele_p, ele_q)
print(f"手写函数求得的解为：x = {res} mod {p * q}")

手写函数求得的解为：x = 77293988037011695380345818 mod 4951760154835678088235319297


In [9]:
# 这个 sympy 库里的 crt模块与 SageMath 里的 crt 功能一致
from sympy.ntheory.modular import crt

remainders = [ele_p, ele_q]  
moduli = [p, q]      

result, mod = crt(moduli, remainders)
print(f"使用标准库计算得到的解为: x ≡ {result} mod {mod}")  

使用标准库计算得到的解为: x ≡ 77293988037011695380345818 mod 4951760154835678088235319297


In [10]:
if res == result:
    print("校验通过")
else:
    print("校验未通过")

校验通过


# ====--------任务二--第二组--------==== #

In [11]:
P = [0x7fffffd8001, 0x7fffffc8001, 0xfffffffc001, 0xffffff6c001, 0xfffffebc001]

print("验证 0x7fffffd8001 是否为素数：", is_prime(P[0]) == True)
print("验证 0x7fffffc8001 是否为素数：", is_prime(P[1]) == True)
print("验证 0xfffffffc001 是否为素数：", is_prime(P[2]) == True)
print("验证 0xffffff6c001 是否为素数：", is_prime(P[3]) == True)
print("验证 0xfffffebc001 是否为素数：", is_prime(P[4]) == True)

验证 0x7fffffd8001 是否为素数： True
验证 0x7fffffc8001 是否为素数： True
验证 0xfffffffc001 是否为素数： True
验证 0xffffff6c001 是否为素数： True
验证 0xfffffebc001 是否为素数： True


### ==----CRT正向变换----== ###

In [12]:
def mod_inverse(a: int, p: int) -> int:
    """计算整数 a 在模 p 下的乘法逆元"""
    a = a % p
    old_r, r = a, p
    old_s, s = 1, 0
    old_t, t = 0, 1
    while r != 0:
        quotient = old_r // r
        old_r, r = r, old_r - quotient * r
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t
    gcd_result = old_r
    if gcd_result != 1:
        raise ValueError(f"逆元不存在，因为 gcd({a}, {p}) = {gcd_result}")
    return old_s % p

In [13]:
X = [0, 0, 0, 0, 0]
for i in range(5):
    X[i] = random_element_from_multiplicative_group(P[i])
    print(f"从 Z*{P[i]} 中随机取的元素为：", X[i])

从 Z*8796092858369 中随机取的元素为： 4710851067614
从 Z*8796092792833 中随机取的元素为： 3314214691814
从 Z*17592186028033 中随机取的元素为： 6226271223395
从 Z*17592185438209 中随机取的元素为： 4190853875178
从 Z*17592184717313 中随机取的元素为： 2649188608357


In [14]:
N = P[0] * P[1] * P[2] * P[3] * P[4]
M = [
    P[1] * P[2] * P[3] * P[4], P[0] * P[2] * P[3] * P[4], P[0] * P[1] * P[3] * P[4], P[0] * P[1] * P[2] * P[4], P[0] * P[1] * P[2] * P[3]
    ]
Inv_M = [0, 0, 0, 0, 0]
for i in range(5):
    if P[i] * M[i] != N:
        raise ValueError("M 校验失败")
print("M 校验成功")

for i in range(5):
    Inv_M[i] = mod_inverse(M[i], P[i])
    
for i in range(5):
    if Inv_M[i] != pow(M[i], -1, P[i]):
        raise ValueError("Inv_M 校验失败")
print("Inv_M 校验成功")

M 校验成功
Inv_M 校验成功


In [15]:
x = 0
for i in range(5):
    x += (X[i] * M[i]) * Inv_M[i]
x = x % N
print("手写函数计算结果为：", x)

手写函数计算结果为： 399444177528579132939443747000962146189636177958820669149303353840


In [16]:
# 这个 sympy 库里的 crt模块与 SageMath 里的 crt 功能一致
from sympy.ntheory.modular import crt

remainders = X
moduli = P    

result_std, mod_std = crt(moduli, remainders)
print(f"使用标准库计算得到的解为: x ≡ {result_std}")  

使用标准库计算得到的解为: x ≡ 399444177528579132939443747000962146189636177958820669149303353840


In [17]:
if x == result_std:
    print("CRT正向变换校验通过")
else:
    print("CRT正向校验未通过")

CRT正向变换校验通过


### ==----CRT逆向变换----== ###

In [18]:
N_ = P[0] * P[1] * P[2] * P[3] * P[4]
x_ = random_element_from_multiplicative_group(N_)
X_ = [0, 0, 0, 0, 0]

In [19]:
for i in range(5):
    X_[i] = x_ % P[i]
    print(f"手写的逆向x{i}为：", X_[i])
print("随机生成的x为：", x_)

手写的逆向x0为： 852259388747
手写的逆向x1为： 3006530703674
手写的逆向x2为： 14627691818775
手写的逆向x3为： 17554529291493
手写的逆向x4为： 11092572945113
随机生成的x为： 345201157169945667684117765066905423209279513651580104815753310241


In [20]:
# 这个 sympy 库里的 crt模块与 SageMath 里的 crt 功能一致
from sympy.ntheory.modular import crt

remainders = X_
moduli = P    

result_std_, mod_std_ = crt(moduli, remainders)
print(f"使用标准库计算得到的解为: x ≡ {result_std_}")  

使用标准库计算得到的解为: x ≡ 345201157169945667684117765066905423209279513651580104815753310241


In [21]:
if x_ == result_std_:
    print("CRT逆向变换校验通过")
else:
    print("CRT逆向校验未通过")

CRT逆向变换校验通过
